<a href="https://colab.research.google.com/github/kavish-24/Konkani_Mental_Health/blob/main/WhiperFineTuneSmall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets evaluate jiwer torch torchaudio accelerate tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.8 MB/s eta 0:00:00


In [ ]:
import os
from transformers import WhisperTokenizer

# Load tokenizer (use whisper-small since you're fine-tuning it)
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")

# Path where your transcripts are stored
transcript_dir = "/content/drive/MyDrive/training/10 Aug"  # <-- change to your directory

for filename in os.listdir(transcript_dir):
    if filename.endswith(".txt"):  # process only text files
        filepath = os.path.join(transcript_dir, filename)

        # Read the transcript file
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read().strip()

        # Tokenize and count
        tokens = tokenizer(text, return_tensors="pt").input_ids
        token_count = tokens.shape[1]

        print(f"{filename}: {token_count} tokens")


News_100817_segment_014_transcript.txt: 418 tokens
News_100817_segment_003_transcript.txt: 521 tokens
News_100817_segment_001_transcript.txt: 421 tokens
News_100817_segment_012_transcript.txt: 441 tokens
News_100817_segment_007_transcript.txt: 406 tokens
News_100817_segment_005_transcript.txt: 403 tokens
News_100817_segment_010_transcript.txt: 249 tokens
News_100817_segment_011_transcript.txt: 401 tokens
News_100817_segment_015_transcript.txt: 92 tokens
News_100817_segment_002_transcript.txt: 47 tokens
News_100817_segment_008_transcript.txt: 395 tokens
News_100817_segment_009_transcript.txt: 135 tokens
News_100817_segment_013_transcript.txt: 388 tokens
News_100817_segment_006_transcript.txt: 440 tokens
News_100817_segment_004_transcript.txt: 242 tokens


In [ ]:
"""
Fine-tune Whisper Small model for Konkani language - Monthly Incremental Training
Trains one month at a time with tracking to prevent retraining
"""

import os
import torch
from datasets import Dataset, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import numpy as np
import librosa
import json
from pathlib import Path
from datetime import datetime

# Configuration
AUDIO_DIR = "/content/drive/MyDrive/Anju Project (1)/Audio_segments"
TRANSCRIPT_DIR = "/content/drive/MyDrive/Anju Project (1)/training_through_bot"
MODEL_NAME = "openai/whisper-small"
OUTPUT_DIR = "/content/drive/MyDrive/whisper-small-konkani"
LANGUAGE = "marathi"
TASK = "transcribe"

# Monthly Training Configuration
TRAINING_TRACKER_FILE = os.path.join(OUTPUT_DIR, "training_progress.json")
CURRENT_MONTH_TO_TRAIN = None  # Set this to train specific month, e.g., "2024-01" or None for next available

# Training parameters
# Training parameters
BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 1e-5
WARMUP_RATIO = 0.1  # 10% of training for warmup
EPOCHS_PER_MONTH = 15  # Train for 15 epochs per month
EVAL_STEPS = 50  # Evaluate more frequently with smaller dataset
SAVE_STEPS = 50  # Save more frequently
SAVE_TOTAL_LIMIT = 3


def load_training_progress():
    """Load the training progress tracker"""
    if os.path.exists(TRAINING_TRACKER_FILE):
        with open(TRAINING_TRACKER_FILE, 'r') as f:
            return json.load(f)
    return {
        "trained_months": [],
        "current_model_path": MODEL_NAME,
        "training_history": []
    }


def save_training_progress(progress):
    """Save the training progress tracker"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(TRAINING_TRACKER_FILE, 'w') as f:
        json.dump(progress, f, indent=2)


def extract_month_from_path(path):
    """
    Extract month from file path
    Handles month name folders: May, June, July, Aug, October, November, HOD
    """
    path_parts = Path(path).parts

    # Month folders in your structure
    month_folders = ['May', 'June', 'July', 'Aug', 'August', 'October', 'November', 'HOD']

    # Find month folder in path
    for part in path_parts:
        if part in month_folders:
            return part

    return None


def get_available_months(audio_dir, transcript_dir):
    """
    Scan directories to find all available months with data
    Returns months in the order: May, June, July, Aug, October, November, HOD
    """
    audio_dir_path = Path(audio_dir)
    audio_files = list(audio_dir_path.rglob("*.mp3"))

    months_data = {}

    for audio_path in audio_files:
        month = extract_month_from_path(audio_path)
        if month:
            if month not in months_data:
                months_data[month] = []
            months_data[month].append(str(audio_path))

    # Define custom sorting order
    month_order = ['May', 'June', 'July', 'Aug', 'August', 'October', 'November', 'HOD']

    # Sort months according to custom order
    sorted_months = [m for m in month_order if m in months_data]

    return sorted_months, months_data


def select_month_to_train(progress):
    """
    Determine which month to train next
    """
    print("\n" + "=" * 70)
    print("MONTHLY TRAINING SYSTEM")
    print("=" * 70)

    # Get available months
    available_months, months_data = get_available_months(AUDIO_DIR, TRANSCRIPT_DIR)

    if not available_months:
        print("ERROR: No months found in the data directories!")
        print("Please check your directory structure.")
        return None

    trained_months = set(progress.get("trained_months", []))
    remaining_months = [m for m in available_months if m not in trained_months]

    print(f"\nAvailable months: {len(available_months)}")
    print(f"Trained months: {len(trained_months)}")
    print(f"Remaining months: {len(remaining_months)}")

    if trained_months:
        print(f"\nAlready trained: {', '.join(sorted(trained_months))}")

    if remaining_months:
        print(f"\nRemaining to train: {', '.join(remaining_months)}")
    else:
        print("\n✓ All months have been trained!")
        return None

    # Determine which month to train
    if CURRENT_MONTH_TO_TRAIN:
        if CURRENT_MONTH_TO_TRAIN in trained_months:
            print(f"\n⚠ WARNING: Month {CURRENT_MONTH_TO_TRAIN} has already been trained!")
            response = input("Do you want to retrain this month? (yes/no): ").strip().lower()
            if response != 'yes':
                print("Training cancelled.")
                return None
            print(f"Proceeding to retrain {CURRENT_MONTH_TO_TRAIN}...")
            return CURRENT_MONTH_TO_TRAIN
        elif CURRENT_MONTH_TO_TRAIN in available_months:
            return CURRENT_MONTH_TO_TRAIN
        else:
            print(f"\nERROR: Month {CURRENT_MONTH_TO_TRAIN} not found in data!")
            return None
    else:
        # Train next available month chronologically
        next_month = remaining_months[0]
        print(f"\n→ Next month to train: {next_month}")
        print(f"   Files available: {len(months_data[next_month])}")

        response = input("\nProceed with training this month? (yes/no): ").strip().lower()
        if response == 'yes':
            return next_month
        else:
            print("Training cancelled.")
            return None


def load_audio_transcript_pairs_for_month(audio_dir, transcript_dir, target_month):
    """
    Load audio files and transcripts for a specific month only
    """
    data = []
    audio_dir_path = Path(audio_dir)
    transcript_dir_path = Path(transcript_dir)

    audio_files = list(audio_dir_path.rglob("*.mp3"))

    print(f"\nScanning for {target_month} data...")

    for audio_path in audio_files:
        month = extract_month_from_path(audio_path)

        if month != target_month:
            continue

        rel_path = audio_path.relative_to(audio_dir_path)
        transcript_rel_path = rel_path.with_suffix('.txt')
        transcript_path = transcript_dir_path / transcript_rel_path

        if transcript_path.exists():
            try:
                with open(transcript_path, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()

                if transcript:
                    data.append({
                        'audio': str(audio_path),
                        'sentence': transcript
                    })
            except Exception as e:
                print(f"Error reading transcript {transcript_path}: {e}")

    print(f"Found {len(data)} audio-transcript pairs for {target_month}")
    return data


def load_and_resample_audio(audio_path, target_sr=16000):
    """Load and resample audio file"""
    try:
        audio_array, sr = librosa.load(audio_path, sr=target_sr)
        return audio_array
    except Exception as e:
        print(f"Error loading {audio_path}: {e}")
        return None


def prepare_dataset(audio_dir, transcript_dir, target_month, test_size=0.1):
    """Prepare dataset for specific month"""
    data = load_audio_transcript_pairs_for_month(audio_dir, transcript_dir, target_month)

    if not data:
        raise ValueError(f"No data found for month {target_month}!")

    dataset = Dataset.from_dict({
        'audio_path': [item['audio'] for item in data],
        'sentence': [item['sentence'] for item in data]
    })

    dataset = dataset.train_test_split(test_size=test_size, seed=42)

    print(f"Train set: {len(dataset['train'])} samples")
    print(f"Test set: {len(dataset['test'])} samples")

    return dataset


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """Data collator for speech-to-text models"""
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


def prepare_data(batch, processor, max_label_length=448):
    """Prepare data for training"""
    audio_array = load_and_resample_audio(batch["audio_path"], target_sr=16000)

    if audio_array is None:
        batch["input_features"] = None
        batch["labels"] = None
        batch["is_valid"] = False
        batch["skip_reason"] = "audio_load_failed"
        return batch

    labels = processor.tokenizer(batch["sentence"]).input_ids

    if len(labels) > max_label_length:
        batch["input_features"] = None
        batch["labels"] = None
        batch["is_valid"] = False
        batch["skip_reason"] = f"labels_too_long_{len(labels)}_tokens"
        return batch

    batch["input_features"] = processor.feature_extractor(
        audio_array, sampling_rate=16000
    ).input_features[0]

    batch["labels"] = labels
    batch["is_valid"] = True
    batch["skip_reason"] = None

    return batch


def compute_metrics(pred, processor, metric):
    """Compute WER metric"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


def main():
    """Main training function with monthly incremental training"""

    # Load training progress
    progress = load_training_progress()

    # Select month to train
    target_month = select_month_to_train(progress)

    if target_month is None:
        print("\nNo training to perform. Exiting.")
        return

    print("\n" + "=" * 70)
    print(f"TRAINING MONTH: {target_month}")
    print("=" * 70)

    # Determine model path (use previous month's model or base model)
    model_path = progress.get("current_model_path", MODEL_NAME)

    print(f"\nStarting from model: {model_path}")

    # Load model and processor
    print("\nLoading model and processor...")
    feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
    tokenizer = WhisperTokenizer.from_pretrained(model_path, language=LANGUAGE, task=TASK)
    processor = WhisperProcessor.from_pretrained(model_path, language=LANGUAGE, task=TASK)

    model = WhisperForConditionalGeneration.from_pretrained(model_path)
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    model.config.use_cache = False
    model.train()
    model.generation_config.language = LANGUAGE
    model.generation_config.task = TASK

    # Prepare dataset for this month
    print("\nPreparing dataset...")
    dataset = prepare_dataset(AUDIO_DIR, TRANSCRIPT_DIR, target_month)

    # Process dataset
    print("\nProcessing audio and text...")
    train_audio_paths = dataset["train"]["audio_path"]
    test_audio_paths = dataset["test"]["audio_path"]

    dataset = dataset.map(
        lambda batch: prepare_data(batch, processor, max_label_length=448),
        remove_columns=dataset["train"].column_names,
        num_proc=1
    )

    # Filter valid samples
    print("\nFiltering valid samples...")
    train_valid_indices = [i for i, valid in enumerate(dataset["train"]["is_valid"]) if valid]
    test_valid_indices = [i for i, valid in enumerate(dataset["test"]["is_valid"]) if valid]

    dataset["train"] = dataset["train"].select(train_valid_indices)
    dataset["test"] = dataset["test"].select(test_valid_indices)
    dataset = dataset.remove_columns(["is_valid", "skip_reason"])

    print(f"\nFiltered dataset:")
    print(f"Train set: {len(dataset['train'])} samples")
    print(f"Test set: {len(dataset['test'])} samples")

    # Data collator and metric
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
    metric = evaluate.load("wer")

    # Output directory for this month
    month_output_dir = os.path.join(OUTPUT_DIR, f"month_{target_month}")

    # Training arguments
    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=month_output_dir,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        warmup_ratio=WARMUP_RATIO,
        num_train_epochs=EPOCHS_PER_MONTH,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        fp16=True,
        eval_strategy="steps",
        per_device_eval_batch_size=BATCH_SIZE,
        predict_with_generate=True,
        generation_max_length=225,
        save_steps=EVAL_STEPS,
        eval_steps=EVAL_STEPS,
        logging_steps=10,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        save_total_limit=SAVE_TOTAL_LIMIT,
        dataloader_num_workers=0,
        remove_unused_columns=False,
    )

    # Trainer
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        data_collator=data_collator,
        compute_metrics=lambda pred: compute_metrics(pred, processor, metric),
        processing_class=processor.feature_extractor,
    )

    print("\n" + "=" * 70)
    print(f"STARTING TRAINING FOR {target_month}")
    print("=" * 70)
    print(f"Epochs: {EPOCHS_PER_MONTH}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
    print(f"Training samples: {len(dataset['train'])}")
    print(f"Steps per epoch: ~{len(dataset['train']) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)}")
    print("=" * 70 + "\n")

    # Train
    trainer.train()

    # Save model
    print("\nSaving model...")
    trainer.save_model(month_output_dir)
    processor.save_pretrained(month_output_dir)

    # Update progress
    if target_month not in progress["trained_months"]:
        progress["trained_months"].append(target_month)

    progress["current_model_path"] = month_output_dir
    progress["training_history"].append({
        "month": target_month,
        "completed_at": datetime.now().isoformat(),
        "model_path": month_output_dir,
        "train_samples": len(dataset["train"]),
        "test_samples": len(dataset["test"])
    })

    save_training_progress(progress)

    print("\n" + "=" * 70)
    print(f"✓ TRAINING COMPLETE FOR {target_month}")
    print(f"Model saved to: {month_output_dir}")
    print(f"Months trained: {len(progress['trained_months'])}")
    print("=" * 70)

    # Show remaining months
    available_months, _ = get_available_months(AUDIO_DIR, TRANSCRIPT_DIR)
    remaining = [m for m in available_months if m not in progress["trained_months"]]

    if remaining:
        print(f"\nRemaining months to train: {', '.join(remaining)}")
        print("Run the script again to train the next month.")
    else:
        print("\n🎉 All months have been trained!")


if __name__ == "__main__":
    main()


MONTHLY TRAINING SYSTEM

Available months: 7
Trained months: 2
Remaining months: 5

Already trained: June, May

Remaining to train: July, Aug, October, November, HOD

→ Next month to train: July
   Files available: 147


KeyboardInterrupt: Interrupted by user

In [ ]:
!apt-get install tree -y
!tree -a "/content/drive/MyDrive/Anju Project (1)/Audio_segments"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/content/drive/MyDrive/Anju Project (1)/Audio_segments
├── Aug
│   ├── Konkani Prime News_070817
│   │   ├── Konkani Prime News_070817_segment_001.mp3
│   │   ├── Konkani Prime News_070817_segment_002.mp3
│   │   ├── Konkani Prime News_070817_segment_003.mp3
│   │   ├── Konkani Prime News_070817_segment_004.mp3
│   │   ├── Konkani Prime News_070817_segment_005.mp3
│   │   ├── Konkani Prime News_070817_segment_006.mp3
│   │   ├── Konkani Prime News_070817_segment_007.mp3
│   │   ├── Konkani Prime News_070817_segment_008.mp3
│   │   ├── Konkani Prime News_070817_segment_009.mp3
│   │   ├── Konkani Prime News_070817_segment_010.mp3
│   │   ├── Konkani Prime News_070817_segment_011.mp3
│   │   ├── Konkani Prime News_070817_segment_012.mp3
│   │   ├── Konkani Prime News_070817_segment_01

In [ ]:
!apt-get install tree -y
!tree -a "/content/drive/MyDrive/Anju Project (1)/training_through_bot"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (54.9 kB/s)
Selecting previously unselected package tree.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
/content/drive/MyDrive/Anju Project (1)/training_through_bot
├── Aug
│   ├── Konkani Prime News_070817
│   │   ├── Konkani Prime News_070817_segment_001.txt
│   │   ├── Konkani Prime News_070817_segment_002.txt
│   │   ├── Konkani Prime News_070817_segment_003.txt
│   │   ├── Konkani Pr

In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

# Load your trained model
model_path = "/content/drive/MyDrive/whisper-small-konkani/month_May"
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.to("cuda")

# Test on a sample audio file
test_audio_path = "/content/drive/MyDrive/Anju Project (1)/Audio_segments/June/konkani prime news_01.0617/konkani prime news_01.0617_segment_002.mp3"

# Load and process audio
audio_array, sr = librosa.load(test_audio_path, sr=16000)
input_features = processor(audio_array, sampling_rate=16000, return_tensors="pt").input_features
input_features = input_features.to("cuda")

# Generate transcription
with torch.no_grad():
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print("Predicted:", transcription)

# Compare with actual transcript
transcript_path = test_audio_path.replace("Audio_segments", "training_through_bot").replace(".mp3", ".txt")
with open(transcript_path, 'r', encoding='utf-8') as f:
    actual = f.read().strip()

print("Actual:   ", actual)

Predicted:  जायका केशींत हाजीर जालो ना मुणा आदलो मुख्यमंत्री दिगंबर कामतीक पण्जे जिल्लो स्शन्स कोर्टान काडले अ जामीन पात्र अटक वारॅंट आदोगादाचे विनोणे उपरांत वारॅंट फाटीं पाकिस्तानान शस्त्रस अन्दी मोडले उपरांत केल्ले फारपेटांत भारती जावाना नी पाकिस्तानाचे पाट जावानाक माल्ले पाकिस्तानाचे सजावान जक्मी राजोरी पुच वाठारांत लचकराची कारवाय.
Actual:    जाइका केशीत हाजीर जालोना मण आदलो मुख्यमंतरी दिगंबर कामतिक पंजे जिल्लो सेशनस कोटान कालने अजामीन पातर अटक वारंट आदोगादाचे विनोवने उपरांत वारंट फाटी पाकिस्तान शस्त्र संदी मोडले उपरांट केले फारपेटांत भारती जवाननी पाकिस्तानाचे पांच जवानाक मलले पाकिस्तानाचे सजवान जक्मी राजोरी पूंच वाठारांत लशकराची कारवा है


In [ ]:
# Add this script to analyze your transcripts
import re
from pathlib import Path
from collections import Counter

TRANSCRIPT_DIR = "/content/drive/MyDrive/Anju Project (1)/training_through_bot"

# Find common words with spelling variations
all_words = []
for txt_file in Path(TRANSCRIPT_DIR).rglob("*.txt"):
    with open(txt_file, 'r', encoding='utf-8') as f:
        words = f.read().strip().split()
        all_words.extend(words)

# Most common words
word_counts = Counter(all_words)
print("Top 20 most common words:")
for word, count in word_counts.most_common(20):
    print(f"{word}: {count}")

# Find similar words (potential spelling inconsistencies)
from difflib import get_close_matches
unique_words = list(word_counts.keys())
print("\n\nPotential spelling inconsistencies:")
checked = set()
for word in unique_words[:100]:  # Check top 100 words
    if word not in checked:
        similar = get_close_matches(word, unique_words, n=5, cutoff=0.8)
        if len(similar) > 1:
            print(f"Similar: {similar}")
        checked.update(similar)

Top 20 most common words:
आनी: 465
म्हूण: 291
एक: 174
ना: 126
ना.: 124
ते: 112
तो: 102
मात: 88
खंय: 81
ह्या: 80
हो: 78
चे: 78
हें: 77
मंत्री: 77
जाल्यार: 72
ही: 67
प्रुडंट: 67
आसा.: 65
ती: 62
आमदार: 61


Potential spelling inconsistencies:
Similar: ['हुशार', 'बहुशकार']
Similar: ['भुरगीं', 'भुरगींय', 'भुरगीं,', 'भुरगी', 'भुरगें']
Similar: ['शिवा', 'शिवाजी']
Similar: ['आनी', 'आनीक', 'आंनी', 'नी', 'आन']
Similar: ['रावतालीं.', 'वतालीं.']
Similar: ['न्हंयचे', 'न्हंयाचे', 'न्हंय', 'न्हयांचे', 'न्हंयचो,']
Similar: ['नुस्तें', 'नुस्तें,', 'नुस्ते', 'नुस्तेंकार']
Similar: ['भाजी', 'भाजी.', 'भाजी,', 'फळभाजी']
Similar: ['रोयताली.', 'करताली.']
Similar: ['तांका', 'तांकाय', 'तांकां', 'ताका', 'तांक']
Similar: ['तीन', 'ती']
Similar: ['आशिल्ली.', 'आशिल्ली', 'आशिल्लो.', 'आशिल्ले.', 'आयिल्ली.']
Similar: ['परेश,', 'परेश', 'परेश…']
Similar: ['नरेश', 'नरेशान']
Similar: ['गुड्डू.', 'गुड्डू…', 'गुड्डूल', 'गुड्डून']
Similar: ['येतकूच', 'येतकच']
Similar: ['तिगांय', 'तिगा']
Similar: ['रानांत', 'रानांतलीं', 'राना

In [ ]:
"""
Check transcript quality by comparing against Whisper predictions
Identifies suspicious/potentially incorrect transcripts
"""

import json
import librosa
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from difflib import SequenceMatcher
from pathlib import Path
from tqdm import tqdm
import gc

# Configuration
AUDIO_DIR = "/content/drive/MyDrive/Anju Project (1)/Audio_segments"
TRANSCRIPT_DIR = "/content/drive/MyDrive/Anju Project (1)/training_through_bot"
OUTPUT_DIR = "/content/drive/MyDrive/quality_check_results"

# IMPORTANT: Set which month to check
TARGET_MONTH = "May"  # Change this to: May, June, July, Aug, October, November, HOD

# Model options:
# Option 1: Use your trained model (RECOMMENDED - faster, optimized for Konkani)
# MODEL_PATH = "/content/drive/MyDrive/whisper-small-konkani/month_May"

# Option 2: Use Whisper base (faster than large, still good quality)
MODEL_PATH = "/content/drive/MyDrive/whisper-small-konkani/month_May"


# Option 3: Use Whisper large-v3 (best quality but very slow)
# MODEL_PATH = "openai/whisper-large-v3"

LANGUAGE = "marathi"  # Language for Konkani transcription
SIMILARITY_THRESHOLD = 0.70  # Flag transcripts with <70% similarity
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)
OUTPUT_FILE = f"{OUTPUT_DIR}/bad_transcripts_{TARGET_MONTH}.json"

print(f"Using device: {DEVICE}")
print(f"Model: {MODEL_PATH}")
print(f"Target month: {TARGET_MONTH}")
print(f"Similarity threshold: {SIMILARITY_THRESHOLD}")

# Load model and processor
print("\nLoading model...")
processor = AutoProcessor.from_pretrained(MODEL_PATH)
model = AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_PATH)
model.to(DEVICE)
model.eval()

# Set language for generation
if hasattr(model.generation_config, 'language'):
    model.generation_config.language = LANGUAGE

def get_similarity(a, b):
    """Calculate string similarity (0-1)"""
    return SequenceMatcher(None, a, b).ratio()

def transcribe_audio(audio_path):
    """Transcribe audio file with error handling"""
    try:
        # Load audio
        audio, sr = librosa.load(str(audio_path), sr=16000)

        # Process
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # Generate prediction
        with torch.no_grad():
            pred_ids = model.generate(**inputs, language=LANGUAGE)

        pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
        return pred_text, None

    except Exception as e:
        return None, str(e)

# Collect all audio files for target month
print(f"\nScanning for {TARGET_MONTH} audio files...")
all_audio_files = list(Path(AUDIO_DIR).rglob("*.mp3"))

# Filter for target month
audio_files = []
for audio_path in all_audio_files:
    if TARGET_MONTH in str(audio_path):
        audio_files.append(audio_path)

print(f"Found {len(audio_files)} audio files in {TARGET_MONTH}")

# Process files
bad_files = []
skipped_files = []
processed = 0

print("\nProcessing files...")
for audio_path in tqdm(audio_files, desc="Checking transcripts"):
    # Find corresponding transcript
    rel_path = audio_path.relative_to(AUDIO_DIR)
    transcript_path = Path(TRANSCRIPT_DIR) / rel_path.with_suffix(".txt")

    if not transcript_path.exists():
        skipped_files.append({
            "audio": str(audio_path),
            "reason": "transcript_not_found"
        })
        continue

    # Load transcript
    try:
        with open(transcript_path, "r", encoding="utf-8") as f:
            label = f.read().strip()

        if not label:
            skipped_files.append({
                "audio": str(audio_path),
                "reason": "empty_transcript"
            })
            continue

    except Exception as e:
        skipped_files.append({
            "audio": str(audio_path),
            "reason": f"transcript_read_error: {e}"
        })
        continue

    # Transcribe with model
    pred_text, error = transcribe_audio(audio_path)

    if error:
        skipped_files.append({
            "audio": str(audio_path),
            "reason": f"audio_processing_error: {error}"
        })
        continue

    # Calculate similarity
    sim = get_similarity(pred_text, label)

    # Flag suspicious transcripts
    if sim < SIMILARITY_THRESHOLD:
        bad_files.append({
            "audio": str(audio_path),
            "transcript_file": str(transcript_path),
            "label": label,
            "model_prediction": pred_text,
            "similarity": round(sim, 3),
            "month": str(audio_path).split('/')[-2] if '/' in str(audio_path) else "unknown"
        })

    processed += 1

    # Periodic cleanup to prevent memory issues
    if processed % 50 == 0:
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Save results
print("\nSaving results...")

# Save bad transcripts
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(bad_files, f, indent=2, ensure_ascii=False)

# Save skipped files
skipped_file = OUTPUT_FILE.replace(".json", "_skipped.json")
with open(skipped_file, "w", encoding="utf-8") as f:
    json.dump(skipped_files, f, indent=2, ensure_ascii=False)

# Summary report
print("\n" + "="*70)
print(f"TRANSCRIPT QUALITY CHECK COMPLETE - {TARGET_MONTH}")
print("="*70)
print(f"Total audio files scanned: {len(audio_files)}")
print(f"Successfully processed: {processed}")
print(f"Skipped (errors/missing): {len(skipped_files)}")
print(f"Suspicious transcripts found: {len(bad_files)}")
print(f"Quality rate: {((processed - len(bad_files)) / processed * 100):.1f}%" if processed > 0 else "N/A")
print(f"\nSimilarity threshold: {SIMILARITY_THRESHOLD*100}%")
print(f"\nResults saved to:")
print(f"  - Bad transcripts: {OUTPUT_FILE}")
print(f"  - Skipped files: {skipped_file}")
print("="*70)

# Show breakdown by month
if bad_files:
    print(f"\nWorst 5 transcripts (lowest similarity):")
    sorted_bad = sorted(bad_files, key=lambda x: x['similarity'])[:5]
    for i, item in enumerate(sorted_bad, 1):
        print(f"\n{i}. Similarity: {item['similarity']*100:.1f}%")
        print(f"   File: {Path(item['audio']).name}")
        print(f"   Label: {item['label'][:80]}...")
        print(f"   Model: {item['model_prediction'][:80]}...")
else:
    print(f"\n✓ All transcripts in {TARGET_MONTH} look good!")

print(f"\n✓ Quality check complete for {TARGET_MONTH}!")
print(f"\nTo check other months, change TARGET_MONTH to:")
print("  'June', 'July', 'Aug', 'October', 'November', or 'HOD'")

Using device: cuda
Model: /content/drive/MyDrive/whisper-small-konkani/month_May
Target month: May
Similarity threshold: 0.7

Loading model...

Scanning for May audio files...
Found 130 audio files in May

Processing files...


Checking transcripts: 100%|██████████| 130/130 [07:33<00:00,  3.49s/it]


Saving results...

TRANSCRIPT QUALITY CHECK COMPLETE - May
Total audio files scanned: 130
Successfully processed: 111
Skipped (errors/missing): 19
Suspicious transcripts found: 75
Quality rate: 32.4%

Similarity threshold: 70.0%

Results saved to:
  - Bad transcripts: /content/drive/MyDrive/quality_check_results/bad_transcripts_May.json
  - Skipped files: /content/drive/MyDrive/quality_check_results/bad_transcripts_May_skipped.json

Worst 5 transcripts (lowest similarity):

1. Similarity: 2.1%
   File: Konk Prime News_040517_segment_015.mp3
   Label: मांद्र्या 15 एमएलडीचो वॉटर प्लांट घालपाचें उतर पीडब्ल्युडी मंत्री सुदिनान दिलां....
   Model:  वाजको सुरू आशील्ली एंपीटीची जनसुनावनी सोपून सोपना आमण लोख सामके बेजाळ देळ वाजको...

2. Similarity: 3.8%
   File: Konk Prime News_040517_segment_009.mp3
   Label: पावोवपाचो निर्णय नुस्तेमारी मंत्री पालयेकारान घेतला. एलईडी आनी बुल ट्रॉलींग बारा...
   Model:  पावच वा सेचेटा वा कंटेनर्स वापर विक्री उठपादन आनी विक्रीक बंद केल्यांत गोंयचे ए...

3. Sim

In [ ]:
import json

bad_path = "/content/drive/MyDrive/quality_check_results/bad_transcripts_May.json"

with open(bad_path, "r", encoding="utf-8") as f:
    bad = json.load(f)

print("Total bad transcripts:", len(bad))
bad[:5]  # Show first 5


Total bad transcripts: 75


[{'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_003.mp3',
  'transcript_file': '/content/drive/MyDrive/Anju Project (1)/training_through_bot/May/Konk Prime News_040517/Konk Prime News_040517_segment_003.txt',
  'label': 'ओला उबेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वोस गोंयकार टॅक्सीवाल्यांनी धरलो मंत्रीआमदारांचो रस्तो विजय म्हण्टा नवी टॅक्नॉलॉजी वापरात जाल्यार काम धंदो साल्वार.दर्यात एलईडी बुल ट्रॉलींग करुन नुस्तें मारपाचो इशू',
  'model_prediction': ' चखईल्यान पएलीं ओलाव बेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वस गोंयकार टॅक्सीवाल्यानी दोल्लो मंत्री आमदारां सोरोसतो विजय म्हण्टा नवी टॅक्नॉलूजी वापरात जाल्यार काम दंदसालो आत. तरीयात एलीडी बुल्ट्रॉलिंग कोरु नुस्ते मारपाचो जाल्या.',
  'similarity': 0.422,
  'month': 'Konk Prime News_040517'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_004.mp3',
  'transcript_file': '/content/drive

In [ ]:
skip_path = "/content/drive/MyDrive/quality_check_results/bad_transcripts_May_skipped.json"

with open(skip_path, "r", encoding="utf-8") as f:
    skipped = json.load(f)

print("Total skipped files:", len(skipped))
skipped[:10]  # Show first 10


Total skipped files: 19


[{'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_001.mp3',
  'reason': 'transcript_not_found'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_002.mp3',
  'reason': 'transcript_not_found'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_005.mp3',
  'reason': 'transcript_not_found'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_007.mp3',
  'reason': 'transcript_not_found'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_008.mp3',
  'reason': 'transcript_not_found'},
 {'audio': '/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517/Konk Prime News_040517_segment_018.mp3',
  're

In [ ]:
# ============================================
# AUDIO + TRANSCRIPT VIEWER (MONTH + DAY)
# ============================================

import json
import os
from pathlib import Path
from IPython.display import Audio, display
import ipywidgets as widgets

AUDIO_ROOT = "/content/drive/MyDrive/Anju Project (1)/Audio_segments"
TRANSCRIPT_ROOT = "/content/drive/MyDrive/Anju Project (1)/training_through_bot"

# ---------------------------------
# Build dropdown lists
# ---------------------------------
months = sorted([d.name for d in Path(AUDIO_ROOT).iterdir() if d.is_dir()])

month_dropdown = widgets.Dropdown(
    options=months,
    description="Select Month:",
    layout=widgets.Layout(width='300px')
)

day_dropdown = widgets.Dropdown(
    options=[],
    description="Select Day Folder:",
    layout=widgets.Layout(width='350px')
)

file_dropdown = widgets.Dropdown(
    options=[],
    description="Select Audio:",
    layout=widgets.Layout(width='500px')
)

output = widgets.Output()


# ---------------------------------
# Update DAY dropdown when month changes
# ---------------------------------
def update_days(change):
    month = change['new']
    day_dirs = sorted([d.name for d in Path(AUDIO_ROOT, month).iterdir() if d.is_dir()])
    day_dropdown.options = day_dirs
    if day_dirs:
        day_dropdown.value = day_dirs[0]

month_dropdown.observe(update_days, names='value')


# ---------------------------------
# Update AUDIO file dropdown when day changes
# ---------------------------------
def update_files(change):
    month = month_dropdown.value
    day = change['new']
    audio_files = sorted([f.name for f in Path(AUDIO_ROOT, month, day).glob("*.mp3")])
    file_dropdown.options = audio_files
    if audio_files:
        file_dropdown.value = audio_files[0]

day_dropdown.observe(update_files, names='value')


# ---------------------------------
# Display audio + transcript
# ---------------------------------
def show_audio_and_transcript(change):
    output.clear_output()

    month = month_dropdown.value
    day = day_dropdown.value
    audio_name = change['new']

    audio_path = f"{AUDIO_ROOT}/{month}/{day}/{audio_name}"
    transcript_path = f"{TRANSCRIPT_ROOT}/{month}/{day}/{audio_name.replace('.mp3', '.txt')}"

    with output:
        print("="*120)
        print("🎧 AUDIO FILE:", audio_path)
        display(Audio(audio_path))
        print("-"*120)

        if os.path.exists(transcript_path):
            print("📝 TRANSCRIPT FILE:", transcript_path)
            print("\n---------------- TRANSCRIPT ----------------\n")
            with open(transcript_path, "r", encoding="utf-8") as f:
                print(f.read())
        else:
            print("❌ Transcript NOT FOUND:", transcript_path)

        print("="*120)

file_dropdown.observe(show_audio_and_transcript, names='value')


# ---------------------------------
# Display the UI
# ---------------------------------
display(month_dropdown)
display(day_dropdown)
display(file_dropdown)
display(output)

# Trigger initial updates
if months:
    month_dropdown.value = months[0]


Dropdown(description='Select Month:', layout=Layout(width='300px'), options=('Aug', 'HOD', 'July', 'June', 'Ma…

Dropdown(description='Select Day Folder:', layout=Layout(width='350px'), options=(), value=None)

Dropdown(description='Select Audio:', layout=Layout(width='500px'), options=(), value=None)

Output()

In [ ]:
# create venv recommended
!pip install -U pip
!pip install git+https://github.com/openai/whisper.git     # whisper
!pip install -U openai-whisper                             # alternative pip package if needed
!pip install transformers librosa torch soundfile pandas tqdm numpy python-Levenshtein rapidfuzz
!pip install git+https://github.com/m-bain/whisperX.git    # whisperx
# whisperx dependencies include: semantic-kernel? it will pull required libs


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-u571ql9q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-u571ql9q
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/m-bain/whisperX.git to /tmp/pip-req-build-tehcgm_8
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperX.git /tmp/pip-req-build-tehcgm_8
  Resolved https://github.com/m-bain/whisperX.git to commit d32ec3e3012ec4c0934f4088424c32f3f038b249
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
"""
verify_and_fix_day.py

Usage:
    python verify_and_fix_day.py \
      --audio-dir "/path/to/audio/segments/day_folder" \
      --transcript-dir "/path/to/transcripts/day_folder" \
      --out "verification_results/day_040517.csv"

It will:
 - iterate all audio files in audio-dir (mp3/wav)
 - load transcript from transcript-dir/<same_name>.txt
 - trim silence, transcribe with Whisper (forced to Marathi 'mr')
 - run whisperx alignment to get word-level timestamps
 - compute robust similarity ignoring a mismatching first word
 - detect half-spoken last word and in many cases replace/remove it from transcript
 - write CSV with corrected_transcript suggestions
"""

import argparse
import os
from pathlib import Path
import json
import math
import csv
import tempfile
import numpy as np
import librosa
from tqdm import tqdm
from rapidfuzz import fuzz
import soundfile as sf

# try importing whisperx alignment pipeline
try:
    import whisper
    import whisperx
    import torch
    WHISPERX_AVAILABLE = True
except Exception:
    WHISPERX_AVAILABLE = False
    try:
        import whisper
    except:
        pass

# -------------------------
# Helpers
# -------------------------
def trim_silence(y, sr, top_db=30, min_silence_len_s=0.25):
    # Use librosa.effects.split to remove leading/trailing silence
    intervals = librosa.effects.split(y, top_db=top_db, frame_length=2048, hop_length=512)
    if len(intervals) == 0:
        return y, 0.0, len(y)/sr
    # keep whole (for small segments, we take the union)
    start = intervals[0,0]
    end = intervals[-1,1]
    trimmed = y[start:end]
    start_s = start / sr
    end_s = end / sr
    return trimmed, start_s, end_s

def normalize_text(s):
    # basic normalization for Devanagari: lowercase not needed; remove punctuation, extra spaces
    import re
    s = s.strip()
    s = re.sub(r"[।,.!?;:—\-–\"'()\\[\\]]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def word_level_similarity(label_words, model_words):
    # compute word-level match percent by fuzzy matching best alignment
    if len(label_words) == 0 and len(model_words) == 0:
        return 1.0
    matches = 0
    total = len(label_words)
    for i, lw in enumerate(label_words):
        # find best model word by fuzzy ratio
        best = 0
        for mw in model_words:
            score = fuzz.ratio(lw, mw)
            if score > best:
                best = score
        # consider >70 as match
        if best >= 70:
            matches += 1
    return matches / (total if total>0 else 1)

def smart_correct_transcript(orig_label, model_text, word_timestamps):
    """
    Heuristics with timestamp-based validation:
     1. Handle silent gaps within segments (already trimmed in audio processing)
     2. If first word mismatches but rest matches well, remove first word from transcript
     3. Detect half-spoken last word using timestamps and remove it from transcript
     4. Use word-level timestamps to validate corrections
    """
    lab = normalize_text(orig_label)
    mod = normalize_text(model_text)
    lab_words = lab.split()
    mod_words = mod.split()

    if len(lab_words) == 0:
        return model_text if model_text else "", []

    corrected = lab_words.copy()
    correction_reasons = []

    # 1) HANDLE FIRST WORD MISMATCH using timestamps and similarity
    if len(lab_words) >= 2 and len(mod_words) >= 1:
        # Compare first words
        first_word_match = fuzz.ratio(lab_words[0], mod_words[0])

        # Compare rest of transcript
        lab_skip1 = " ".join(lab_words[1:])
        mod_text = " ".join(mod_words)
        mod_skip1 = " ".join(mod_words[1:]) if len(mod_words) > 1 else mod_text

        skip1_score = fuzz.ratio(lab_skip1, mod_skip1)
        full_score = fuzz.ratio(lab, mod_text)

        # If first word doesn't match well but rest does, remove first word
        if first_word_match < 70 and skip1_score >= 80 and skip1_score > full_score + 10:
            corrected = lab_words[1:]
            correction_reasons.append(f"Removed mismatched first word: '{lab_words[0]}'")

    # 2) HANDLE HALF-SPOKEN LAST WORD using timestamps
    if word_timestamps and len(word_timestamps) > 0 and len(corrected) > 0:
        last_timestamp = word_timestamps[-1]
        last_word_dur = last_timestamp['end'] - last_timestamp['start']
        last_model_word = last_timestamp['word'].strip()
        last_label_word = corrected[-1]

        # Calculate average word duration for comparison
        avg_dur = 0.0
        if len(word_timestamps) > 1:
            total_dur = sum(w['end'] - w['start'] for w in word_timestamps[:-1])
            avg_dur = total_dur / (len(word_timestamps) - 1)

        # Detect half-spoken: very short duration OR poor match with label
        last_word_match = fuzz.ratio(last_label_word, last_model_word)
        is_half_spoken = False

        # Criteria for half-spoken word:
        # - Duration < 0.1s (very short)
        # - Duration < 40% of average word duration
        # - Or very short word length (1-2 chars) with short duration
        # - Or poor match with transcript's last word
        if last_word_dur < 0.1:
            is_half_spoken = True
            correction_reasons.append(f"Last word duration too short: {last_word_dur:.3f}s")
        elif avg_dur > 0 and last_word_dur < 0.4 * avg_dur:
            is_half_spoken = True
            correction_reasons.append(f"Last word duration {last_word_dur:.3f}s << avg {avg_dur:.3f}s")
        elif len(last_model_word) <= 2 and last_word_dur < 0.15:
            is_half_spoken = True
            correction_reasons.append(f"Last word too short: '{last_model_word}' ({last_word_dur:.3f}s)")
        elif last_word_match < 60:
            is_half_spoken = True
            correction_reasons.append(f"Last word mismatch: '{last_label_word}' vs '{last_model_word}' (score: {last_word_match})")

        if is_half_spoken and len(corrected) > 0:
            removed_word = corrected[-1]
            corrected = corrected[:-1]
            correction_reasons.append(f"Removed half-spoken last word: '{removed_word}'")

    # 3) VALIDATE: if corrected version is empty or too different, use model transcript
    corrected_text = " ".join(corrected).strip()

    if corrected_text == "":
        # If all words were removed, use model transcript as fallback
        corrected_text = model_text if model_text else normalize_text(orig_label)
        correction_reasons.append("Used model transcript as fallback")

    return corrected_text, correction_reasons

# -------------------------
# Core processing function
# -------------------------
def process_file(audio_path, transcript_path, model, device="cuda", force_lang="mr"):
    # load audio
    # prefer librosa to get float32 mono
    y, sr = librosa.load(str(audio_path), sr=16000, mono=True)
    trimmed, start_s, end_s = trim_silence(y, sr, top_db=30)
    # if trimmed length very small, use original
    if len(trimmed) < 1600:
        trimmed = y
        start_s, end_s = 0.0, len(y)/sr

    # write tmp trimmed wav for whisperx if needed
    tmp_wav = None
    tmp_dir = tempfile.mkdtemp()
    tmp_wav = os.path.join(tmp_dir, "trimmed.wav")
    sf.write(tmp_wav, trimmed, sr, subtype='PCM_16')

    # 1) transcribe with whisper, forcing Marathi language to avoid script drift
    # Use whisperx if available to get word timestamps
    model_transcript = ""
    word_timestamps = []
    try:
        if WHISPERX_AVAILABLE:
            w_model = whisper.load_model("small")  # small for speed; change to "medium"/"large" if you can
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()

            # Try whisperx alignment (may not work for all languages)
            try:
                device = "cuda" if torch.cuda.is_available() else "cpu"
                # load alignment model and run
                model_a, metadata = whisperx.load_align_model(language_code=force_lang, device=device)
                aligned = whisperx.align(result["segments"], model_a, metadata, tmp_wav, device)
                # aligned['word_segments'] contains list of dicts: word, start, end, score
                word_timestamps = [{"word": w["word"].strip(), "start": w["start"], "end": w["end"]} for w in aligned.get("word_segments", [])]
            except Exception as align_error:
                # Alignment failed (e.g., no model for this language), fall back to segment-based timestamps
                print(f"  Note: Word-level alignment not available for '{force_lang}', using segment timestamps")
                for seg in result.get("segments", []):
                    words = seg["text"].strip().split()
                    seg_start = seg["start"]
                    seg_end = seg["end"]
                    if len(words) == 0:
                        continue
                    dur = (seg_end - seg_start) / len(words)
                    for i, w in enumerate(words):
                        word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})
        else:
            # fallback: whisper only (no word-level timestamps)
            w_model = whisper.load_model("small")
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()
            # If no word timestamps, approximate by segment starts/ends
            word_timestamps = []
            # segments available in result.get("segments", [])
            for seg in result.get("segments", []):
                # split segment into words evenly (approx)
                words = seg["text"].strip().split()
                seg_start = seg["start"]
                seg_end = seg["end"]
                if len(words) == 0:
                    continue
                dur = (seg_end - seg_start) / len(words)
                for i, w in enumerate(words):
                    word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})

    except Exception as e:
        # fallback single whisper call
        try:
            w_model = whisper.load_model("small")
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()
            # Use segment-based word timestamps
            for seg in result.get("segments", []):
                words = seg["text"].strip().split()
                seg_start = seg["start"]
                seg_end = seg["end"]
                if len(words) > 0:
                    dur = (seg_end - seg_start) / len(words)
                    for i, w in enumerate(words):
                        word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})
        except Exception as e2:
            model_transcript = ""
            print(f"  Transcription error: {e2}")
            word_timestamps = []

    # read original transcript
    if transcript_path.exists():
        orig = transcript_path.read_text(encoding="utf-8").strip()
    else:
        orig = ""

    # SKIP FILES WITH NO TRANSCRIPT
    if not orig:
        return None

    # normalize and compare
    lab_norm = normalize_text(orig)
    mod_norm = normalize_text(model_transcript)

    # build list of words for word-level similarity
    lab_words = lab_norm.split()
    mod_words = mod_norm.split()

    # compute robust word-level similarity ignoring first word heuristically:
    sim_full = word_level_similarity(lab_words, mod_words)
    # simulate skipping first label word:
    sim_skip1 = 0.0
    if len(lab_words) > 1:
        sim_skip1 = word_level_similarity(lab_words[1:], mod_words)

    # choose which similarity to use and possibly record correction
    use_skip_first = False
    if sim_skip1 - sim_full >= 0.25:  # skipping first word improved match massively
        use_skip_first = True
        chosen_sim = sim_skip1
    else:
        chosen_sim = sim_full

    # apply smart correction heuristics
    corrected, correction_reasons = smart_correct_transcript(orig, model_transcript, word_timestamps)

    # compute a final fuzzy score on corrected vs model
    corrected_norm = normalize_text(corrected)
    final_word_list = corrected_norm.split()
    final_sim = word_level_similarity(final_word_list, mod_words)

    # package result
    result = {
        "audio": str(audio_path),
        "trim_start_s": float(start_s),
        "trim_end_s": float(end_s),
        "original_transcript": orig,
        "model_transcript": model_transcript,
        "word_timestamps": word_timestamps,
        "sim_full": float(sim_full),
        "sim_skip1": float(sim_skip1),
        "used_skip_first": bool(use_skip_first),
        "chosen_sim": float(chosen_sim),
        "corrected_transcript": corrected,
        "final_similarity": float(final_sim),
        "correction_reasons": " | ".join(correction_reasons) if correction_reasons else "No corrections needed",
        "was_corrected": bool(corrected != orig)
    }
    return result

# -------------------------
# CLI / batch driver
# -------------------------
def main():
    parser = argparse.ArgumentParser(description="Verify and fix transcripts for a single day's audio segments")

    # HARDCODED PATHS - Change these to your actual paths
    parser.add_argument("--audio-dir",
                        default="/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517",
                        help="Directory containing audio files (mp3/wav) for a single day")

    parser.add_argument("--transcript-dir",
                        default="/content/drive/MyDrive/Anju Project (1)/training_through_bot/May/Konk Prime News_040517",
                        help="Directory containing transcript files (.txt) for the same day")

    parser.add_argument("--out",
                        default="/content/drive/MyDrive/verification_results/output.csv",
                        help="Output CSV file path for verification results")

    parser.add_argument("--force-lang", default="mr",
                        help="Language code for Whisper (default: mr for Marathi)")

    # FIX FOR GOOGLE COLAB: Ignore unknown arguments (like Colab's -f flag)
    args, unknown = parser.parse_known_args()

    audio_dir = Path(args.audio_dir)
    transcript_dir = Path(args.transcript_dir)
    out_csv = Path(args.out)
    force_lang = args.force_lang

    # Validate directories exist
    if not audio_dir.exists():
        print(f"Error: Audio directory does not exist: {audio_dir}")
        return

    if not transcript_dir.exists():
        print(f"Error: Transcript directory does not exist: {transcript_dir}")
        return

    # Get all audio files
    audio_paths = sorted(list(audio_dir.glob("*.mp3")) + list(audio_dir.glob("*.wav")))

    if len(audio_paths) == 0:
        print(f"Error: No audio files found in {audio_dir}")
        return

    # Map audio files to their corresponding transcript files
    transcripts = {}
    for audio_path in audio_paths:
        transcript_path = transcript_dir / audio_path.with_suffix(".txt").name
        transcripts[str(audio_path)] = transcript_path

    print(f"Found {len(audio_paths)} audio files in: {audio_dir}")
    print(f"Looking for transcripts in: {transcript_dir}")
    print(f"Output will be saved to: {out_csv}\n")

    # Process all files
    results = []
    skipped_count = 0
    for audio_path in tqdm(audio_paths, desc="Processing audio files"):
        transcript_path = transcripts[str(audio_path)]

        if not transcript_path.exists():
            print(f"\nWarning: Transcript not found for {audio_path.name}, skipping...")
            skipped_count += 1
            continue

        res = process_file(audio_path, transcript_path, model=None, device="cuda", force_lang=force_lang)

        # Skip if no transcript was found (returns None)
        if res is None:
            skipped_count += 1
            continue

        results.append(res)

    # Save results to CSV
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    with open(out_csv, "w", encoding="utf-8", newline="") as csvf:
        writer = csv.DictWriter(csvf, fieldnames=[
            "audio", "trim_start_s", "trim_end_s",
            "original_transcript", "corrected_transcript",
            "model_transcript", "was_corrected", "correction_reasons",
            "chosen_sim", "final_similarity", "used_skip_first"
        ])
        writer.writeheader()

        for r in results:
            writer.writerow({
                "audio": r["audio"],
                "trim_start_s": r["trim_start_s"],
                "trim_end_s": r["trim_end_s"],
                "original_transcript": r["original_transcript"],
                "corrected_transcript": r["corrected_transcript"],
                "model_transcript": r["model_transcript"],
                "was_corrected": r["was_corrected"],
                "correction_reasons": r["correction_reasons"],
                "chosen_sim": r["chosen_sim"],
                "final_similarity": r["final_similarity"],
                "used_skip_first": r["used_skip_first"]
            })

    print(f"\n✓ Saved CSV with {len(results)} results: {out_csv}")

    if skipped_count > 0:
        print(f"  ⚠ Skipped {skipped_count} files with no transcript")

    # Print summary statistics
    corrected_count = sum(1 for r in results if r["was_corrected"])
    print(f"  - {corrected_count}/{len(results)} transcripts were corrected")
    if len(results) > 0:
        print(f"  - Average final similarity: {sum(r['final_similarity'] for r in results)/len(results):.2%}")

    # Show examples of corrections
    corrections = [r for r in results if r["was_corrected"]]
    if corrections:
        print(f"\n📝 Example corrections:")
        for r in corrections[:3]:  # Show first 3
            filename = Path(r["audio"]).name
            print(f"  • {filename}")
            print(f"    Original: {r['original_transcript'][:60]}...")
            print(f"    Corrected: {r['corrected_transcript'][:60]}...")
            print(f"    Reason: {r['correction_reasons']}")
            print()

if __name__ == "__main__":
    main()

Found 21 audio files in: /content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517
Looking for transcripts in: /content/drive/MyDrive/Anju Project (1)/training_through_bot/May/Konk Prime News_040517
Output will be saved to: /content/drive/MyDrive/verification_results/output.csv



Processing audio files:   0%|          | 0/21 [00:00<?, ?it/s]



2025-11-21 16:49:40 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files:  14%|█▍        | 3/21 [00:59<05:55, 19.75s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 16:50:01 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


...

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 16:53:39 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files:  95%|█████████▌| 20/21 [04:57<00:15, 15.32s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 16:53:57 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files: 100%|██████████| 21/21 [05:16<00:00, 15.05s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps

✓ Saved CSV with 15 results: /content/drive/MyDrive/verification_results/output.csv
  ⚠ Skipped 6 files with no transcript
  - 13/15 transcripts were corrected
  - Average final similarity: 8.33%

📝 Example corrections:
  • Konk Prime News_040517_segment_003.mp3
    Original: ओला उबेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वोस गोंयकार टॅ...
    Corrected: ओला उबेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वोस गोंयकार टॅ...
    Reason: Last word mismatch: 'इशू' vs 'मर्बात।' (score: 0.0) | Removed half-spoken last word: 'इशू'

  • Konk Prime News_040517_segment_004.mp3
    Original: सरकार व्हरतलो केंद्रामेरेन एक्शन घेवप गोंयचे अधिकारकक्षेंत न...
    Corrected: सरकार व्हरतलो केंद्रामेरेन एक्शन घेवप गोंयचे अधिकारकक्षेंत न...
    Reason: Last word mismatch: 'खबरां' vs 'election' (score: 0.0) | Removed half-spoken last word: 'खबरां'

  • Konk Prime News_040517_segment_006.mp3
    Original: पंजाब पोलिसांक पयले जैत....
   

In [ ]:
!python verify_and_fix_day.py \
  --audio-root "/content/drive/MyDrive/Anju Project (1)/Audio_segments" \
  --text-root "/content/drive/MyDrive/Anju Project (1)/training_through_bot" \
  --month "May" \
  --day "Konk Prime News_040517" \
  --out "/content/drive/MyDrive/verification_results/day_040517.csv"


python3: can't open file '/content/verify_and_fix_day.py': [Errno 2] No such file or directory


In [ ]:
!python /content/verify_and_fix_day.py


python3: can't open file '/content/verify_and_fix_day.py': [Errno 2] No such file or directory


In [ ]:
"""
verify_and_fix_day.py

Usage:
    python verify_and_fix_day.py \
      --audio-dir "/path/to/audio/segments/day_folder" \
      --transcript-dir "/path/to/transcripts/day_folder" \
      --out "verification_results/day_040517.csv"

It will:
 - iterate all audio files in audio-dir (mp3/wav)
 - load transcript from transcript-dir/<same_name>.txt
 - trim silence, transcribe with Whisper (forced to Marathi 'mr')
 - run whisperx alignment to get word-level timestamps
 - compute robust similarity ignoring a mismatching first word
 - detect half-spoken last word and in many cases replace/remove it from transcript
 - write CSV with corrected_transcript suggestions
"""

import argparse
import os
from pathlib import Path
import json
import math
import csv
import tempfile
import numpy as np
import librosa
from tqdm import tqdm
from rapidfuzz import fuzz
import soundfile as sf

# try importing whisperx alignment pipeline
try:
    import whisper
    import whisperx
    import torch
    WHISPERX_AVAILABLE = True
except Exception:
    WHISPERX_AVAILABLE = False
    try:
        import whisper
    except:
        pass

# -------------------------
# Helpers
# -------------------------
def trim_silence(y, sr, top_db=30, min_silence_len_s=0.25):
    # Use librosa.effects.split to remove leading/trailing silence
    intervals = librosa.effects.split(y, top_db=top_db, frame_length=2048, hop_length=512)
    if len(intervals) == 0:
        return y, 0.0, len(y)/sr
    # keep whole (for small segments, we take the union)
    start = intervals[0,0]
    end = intervals[-1,1]
    trimmed = y[start:end]
    start_s = start / sr
    end_s = end / sr
    return trimmed, start_s, end_s

def normalize_text(s):
    # basic normalization for Devanagari: lowercase not needed; remove punctuation, extra spaces
    import re
    s = s.strip()
    s = re.sub(r"[।,.!?;:—\-–\"'()\\[\\]]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def word_level_similarity(label_words, model_words):
    # compute word-level match percent by fuzzy matching best alignment
    if len(label_words) == 0 and len(model_words) == 0:
        return 1.0
    matches = 0
    total = len(label_words)
    for i, lw in enumerate(label_words):
        # find best model word by fuzzy ratio
        best = 0
        for mw in model_words:
            score = fuzz.ratio(lw, mw)
            if score > best:
                best = score
        # consider >70 as match
        if best >= 70:
            matches += 1
    return matches / (total if total>0 else 1)

def smart_correct_transcript(orig_label, model_text, word_timestamps):
    """
    Heuristics with timestamp-based validation:
     1. Handle silent gaps within segments (already trimmed in audio processing)
     2. If first word mismatches but rest matches well, remove first word from transcript
     3. Detect half-spoken last word using timestamps and remove it from transcript
     4. Use word-level timestamps to validate corrections
     5. Return audio trim points to match the corrected transcript
    """
    lab = normalize_text(orig_label)
    mod = normalize_text(model_text)
    lab_words = lab.split()
    mod_words = mod.split()

    if len(lab_words) == 0:
        return model_text if model_text else "", [], None, None

    corrected = lab_words.copy()
    correction_reasons = []
    audio_trim_start = None  # Will store the time to start audio from
    audio_trim_end = None    # Will store the time to end audio at

    # 1) HANDLE FIRST WORD MISMATCH using timestamps and similarity
    words_removed_from_start = 0
    if len(lab_words) >= 2 and len(mod_words) >= 1:
        # Compare first words
        first_word_match = fuzz.ratio(lab_words[0], mod_words[0])

        # Compare rest of transcript
        lab_skip1 = " ".join(lab_words[1:])
        mod_text = " ".join(mod_words)
        mod_skip1 = " ".join(mod_words[1:]) if len(mod_words) > 1 else mod_text

        skip1_score = fuzz.ratio(lab_skip1, mod_skip1)
        full_score = fuzz.ratio(lab, mod_text)

        # If first word doesn't match well but rest does, remove first word
        if first_word_match < 70 and skip1_score >= 80 and skip1_score > full_score + 10:
            corrected = lab_words[1:]
            words_removed_from_start = 1
            correction_reasons.append(f"Removed mismatched first word: '{lab_words[0]}'")

            # Set audio trim start to after the first word
            if word_timestamps and len(word_timestamps) > 0:
                audio_trim_start = word_timestamps[0]['end']

    # 2) HANDLE HALF-SPOKEN LAST WORD using timestamps
    words_removed_from_end = 0
    if word_timestamps and len(word_timestamps) > 0 and len(corrected) > 0:
        last_timestamp = word_timestamps[-1]
        last_word_dur = last_timestamp['end'] - last_timestamp['start']
        last_model_word = last_timestamp['word'].strip()
        last_label_word = corrected[-1]

        # Calculate average word duration for comparison
        avg_dur = 0.0
        if len(word_timestamps) > 1:
            total_dur = sum(w['end'] - w['start'] for w in word_timestamps[:-1])
            avg_dur = total_dur / (len(word_timestamps) - 1)

        # Detect half-spoken: very short duration OR poor match with label
        last_word_match = fuzz.ratio(last_label_word, last_model_word)
        is_half_spoken = False

        # Criteria for half-spoken word:
        if last_word_dur < 0.1:
            is_half_spoken = True
            correction_reasons.append(f"Last word duration too short: {last_word_dur:.3f}s")
        elif avg_dur > 0 and last_word_dur < 0.4 * avg_dur:
            is_half_spoken = True
            correction_reasons.append(f"Last word duration {last_word_dur:.3f}s << avg {avg_dur:.3f}s")
        elif len(last_model_word) <= 2 and last_word_dur < 0.15:
            is_half_spoken = True
            correction_reasons.append(f"Last word too short: '{last_model_word}' ({last_word_dur:.3f}s)")
        elif last_word_match < 60:
            is_half_spoken = True
            correction_reasons.append(f"Last word mismatch: '{last_label_word}' vs '{last_model_word}' (score: {last_word_match})")

        if is_half_spoken and len(corrected) > 0:
            removed_word = corrected[-1]
            corrected = corrected[:-1]
            words_removed_from_end = 1
            correction_reasons.append(f"Removed half-spoken last word: '{removed_word}'")

            # Set audio trim end to before the last word
            if len(word_timestamps) > 1:
                audio_trim_end = word_timestamps[-2]['end']
            else:
                audio_trim_end = word_timestamps[-1]['start']

    # 3) Calculate audio trim points based on which words to keep
    if word_timestamps and len(word_timestamps) > 0:
        # If we removed words from start, trim audio from the next word's start
        if audio_trim_start is None and words_removed_from_start > 0 and len(word_timestamps) > words_removed_from_start:
            audio_trim_start = word_timestamps[words_removed_from_start]['start']

        # If we removed words from end, trim audio to the last kept word's end
        if audio_trim_end is None and words_removed_from_end > 0:
            kept_words = len(word_timestamps) - words_removed_from_end
            if kept_words > 0:
                audio_trim_end = word_timestamps[kept_words - 1]['end']

    # 4) VALIDATE: if corrected version is empty or too different, use model transcript
    corrected_text = " ".join(corrected).strip()

    if corrected_text == "":
        # If all words were removed, use model transcript as fallback
        corrected_text = model_text if model_text else normalize_text(orig_label)
        correction_reasons.append("Used model transcript as fallback")
        audio_trim_start = None
        audio_trim_end = None

    return corrected_text, correction_reasons, audio_trim_start, audio_trim_end

# -------------------------
# Core processing function
# -------------------------
def process_file(audio_path, transcript_path, model, device="cuda", force_lang="mr", output_audio_dir=None):
    # load audio
    # prefer librosa to get float32 mono
    y, sr = librosa.load(str(audio_path), sr=16000, mono=True)
    original_duration = len(y) / sr

    trimmed, start_s, end_s = trim_silence(y, sr, top_db=30)
    # if trimmed length very small, use original
    if len(trimmed) < 1600:
        trimmed = y
        start_s, end_s = 0.0, len(y)/sr

    # write tmp trimmed wav for whisperx if needed
    tmp_wav = None
    tmp_dir = tempfile.mkdtemp()
    tmp_wav = os.path.join(tmp_dir, "trimmed.wav")
    sf.write(tmp_wav, trimmed, sr, subtype='PCM_16')

    # 1) transcribe with whisper, forcing Marathi language to avoid script drift
    # Use whisperx if available to get word timestamps
    model_transcript = ""
    word_timestamps = []
    try:
        if WHISPERX_AVAILABLE:
            w_model = whisper.load_model("small")  # small for speed; change to "medium"/"large" if you can
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()

            # Try whisperx alignment (may not work for all languages)
            try:
                device = "cuda" if torch.cuda.is_available() else "cpu"
                # load alignment model and run
                model_a, metadata = whisperx.load_align_model(language_code=force_lang, device=device)
                aligned = whisperx.align(result["segments"], model_a, metadata, tmp_wav, device)
                # aligned['word_segments'] contains list of dicts: word, start, end, score
                word_timestamps = [{"word": w["word"].strip(), "start": w["start"], "end": w["end"]} for w in aligned.get("word_segments", [])]
            except Exception as align_error:
                # Alignment failed (e.g., no model for this language), fall back to segment-based timestamps
                print(f"  Note: Word-level alignment not available for '{force_lang}', using segment timestamps")
                for seg in result.get("segments", []):
                    words = seg["text"].strip().split()
                    seg_start = seg["start"]
                    seg_end = seg["end"]
                    if len(words) == 0:
                        continue
                    dur = (seg_end - seg_start) / len(words)
                    for i, w in enumerate(words):
                        word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})
        else:
            # fallback: whisper only (no word-level timestamps)
            w_model = whisper.load_model("small")
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()
            # If no word timestamps, approximate by segment starts/ends
            word_timestamps = []
            # segments available in result.get("segments", [])
            for seg in result.get("segments", []):
                # split segment into words evenly (approx)
                words = seg["text"].strip().split()
                seg_start = seg["start"]
                seg_end = seg["end"]
                if len(words) == 0:
                    continue
                dur = (seg_end - seg_start) / len(words)
                for i, w in enumerate(words):
                    word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})

    except Exception as e:
        # fallback single whisper call
        try:
            w_model = whisper.load_model("small")
            result = w_model.transcribe(tmp_wav, language=force_lang, task="transcribe")
            model_transcript = result["text"].strip()
            # Use segment-based word timestamps
            for seg in result.get("segments", []):
                words = seg["text"].strip().split()
                seg_start = seg["start"]
                seg_end = seg["end"]
                if len(words) > 0:
                    dur = (seg_end - seg_start) / len(words)
                    for i, w in enumerate(words):
                        word_timestamps.append({"word": w.strip(), "start": seg_start + i*dur, "end": seg_start + (i+1)*dur})
        except Exception as e2:
            model_transcript = ""
            print(f"  Transcription error: {e2}")
            word_timestamps = []

    # read original transcript
    if transcript_path.exists():
        orig = transcript_path.read_text(encoding="utf-8").strip()
    else:
        orig = ""

    # SKIP FILES WITH NO TRANSCRIPT
    if not orig:
        return None

    # normalize and compare
    lab_norm = normalize_text(orig)
    mod_norm = normalize_text(model_transcript)

    # build list of words for word-level similarity
    lab_words = lab_norm.split()
    mod_words = mod_norm.split()

    # compute robust word-level similarity ignoring first word heuristically:
    sim_full = word_level_similarity(lab_words, mod_words)
    # simulate skipping first label word:
    sim_skip1 = 0.0
    if len(lab_words) > 1:
        sim_skip1 = word_level_similarity(lab_words[1:], mod_words)

    # choose which similarity to use and possibly record correction
    use_skip_first = False
    if sim_skip1 - sim_full >= 0.25:  # skipping first word improved match massively
        use_skip_first = True
        chosen_sim = sim_skip1
    else:
        chosen_sim = sim_full

    # apply smart correction heuristics
    corrected, correction_reasons, audio_trim_start, audio_trim_end = smart_correct_transcript(orig, model_transcript, word_timestamps)

    # TRIM AND SAVE AUDIO if corrections were made
    final_audio_path = None
    actual_trim_start = start_s
    actual_trim_end = end_s

    if output_audio_dir and (audio_trim_start is not None or audio_trim_end is not None):
        # Calculate actual trim points (relative to original audio)
        # word timestamps are relative to the trimmed audio, so add the initial silence trim offset
        if audio_trim_start is not None:
            actual_trim_start = start_s + audio_trim_start

        if audio_trim_end is not None:
            actual_trim_end = start_s + audio_trim_end
        else:
            actual_trim_end = end_s

        # Convert time to samples
        trim_start_sample = int(actual_trim_start * sr)
        trim_end_sample = int(actual_trim_end * sr)

        # Extract the final audio segment
        final_audio = y[trim_start_sample:trim_end_sample]

        # Save the trimmed audio - always save as WAV for compatibility
        output_audio_dir.mkdir(parents=True, exist_ok=True)
        final_audio_path = output_audio_dir / (audio_path.stem + '.wav')
        sf.write(str(final_audio_path), final_audio, sr, subtype='PCM_16')

        correction_reasons.append(f"Audio trimmed: {actual_trim_start:.2f}s to {actual_trim_end:.2f}s (from {original_duration:.2f}s)")
    elif output_audio_dir and corrected != orig:
        # Even if no word-level trimming, save the silence-trimmed version
        final_audio = trimmed
        output_audio_dir.mkdir(parents=True, exist_ok=True)
        final_audio_path = output_audio_dir / (audio_path.stem + '.wav')
        sf.write(str(final_audio_path), final_audio, sr, subtype='PCM_16')
        correction_reasons.append(f"Audio trimmed (silence only): {start_s:.2f}s to {end_s:.2f}s")

    # compute a final fuzzy score on corrected vs model
    corrected_norm = normalize_text(corrected)
    final_word_list = corrected_norm.split()
    final_sim = word_level_similarity(final_word_list, mod_words)

    # package result
    result = {
        "audio": str(audio_path),
        "trimmed_audio": str(final_audio_path) if final_audio_path else "",
        "original_duration_s": float(original_duration),
        "trim_start_s": float(actual_trim_start),
        "trim_end_s": float(actual_trim_end),
        "final_duration_s": float(actual_trim_end - actual_trim_start),
        "original_transcript": orig,
        "model_transcript": model_transcript,
        "word_timestamps": word_timestamps,
        "sim_full": float(sim_full),
        "sim_skip1": float(sim_skip1),
        "used_skip_first": bool(use_skip_first),
        "chosen_sim": float(chosen_sim),
        "corrected_transcript": corrected,
        "final_similarity": float(final_sim),
        "correction_reasons": " | ".join(correction_reasons) if correction_reasons else "No corrections needed",
        "was_corrected": bool(corrected != orig),
        "was_audio_trimmed": bool(final_audio_path is not None)
    }
    return result

# -------------------------
# CLI / batch driver
# -------------------------
def main():
    parser = argparse.ArgumentParser(description="Verify and fix transcripts for a single day's audio segments")

    # HARDCODED PATHS - Change these to your actual paths
    parser.add_argument("--audio-dir",
                        default="/content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517",
                        help="Directory containing audio files (mp3/wav) for a single day")

    parser.add_argument("--transcript-dir",
                        default="/content/drive/MyDrive/Anju Project (1)/training_through_bot/May/Konk Prime News_040517",
                        help="Directory containing transcript files (.txt) for the same day")

    parser.add_argument("--out",
                        default="/content/drive/MyDrive/verification_results/output.csv",
                        help="Output CSV file path for verification results")

    parser.add_argument("--output-audio-dir",
                        default="/content/drive/MyDrive/verification_results/trimmed_audio",
                        help="Directory to save trimmed audio files")

    parser.add_argument("--force-lang", default="mr",
                        help="Language code for Whisper (default: mr for Marathi)")

    # FIX FOR GOOGLE COLAB: Ignore unknown arguments (like Colab's -f flag)
    args, unknown = parser.parse_known_args()

    audio_dir = Path(args.audio_dir)
    transcript_dir = Path(args.transcript_dir)
    out_csv = Path(args.out)
    output_audio_dir = Path(args.output_audio_dir) if args.output_audio_dir else None
    force_lang = args.force_lang

    # Validate directories exist
    if not audio_dir.exists():
        print(f"Error: Audio directory does not exist: {audio_dir}")
        return

    if not transcript_dir.exists():
        print(f"Error: Transcript directory does not exist: {transcript_dir}")
        return

    # Get all audio files
    audio_paths = sorted(list(audio_dir.glob("*.mp3")) + list(audio_dir.glob("*.wav")))

    if len(audio_paths) == 0:
        print(f"Error: No audio files found in {audio_dir}")
        return

    # Map audio files to their corresponding transcript files
    transcripts = {}
    for audio_path in audio_paths:
        transcript_path = transcript_dir / audio_path.with_suffix(".txt").name
        transcripts[str(audio_path)] = transcript_path

    print(f"Found {len(audio_paths)} audio files in: {audio_dir}")
    print(f"Looking for transcripts in: {transcript_dir}")
    print(f"Output will be saved to: {out_csv}")
    if output_audio_dir:
        print(f"Trimmed audio will be saved to: {output_audio_dir}\n")
    else:
        print("Trimmed audio will NOT be saved (no --output-audio-dir specified)\n")

    # Process all files
    results = []
    skipped_count = 0
    for audio_path in tqdm(audio_paths, desc="Processing audio files"):
        transcript_path = transcripts[str(audio_path)]

        if not transcript_path.exists():
            print(f"\nWarning: Transcript not found for {audio_path.name}, skipping...")
            skipped_count += 1
            continue

        res = process_file(audio_path, transcript_path, model=None, device="cuda", force_lang=force_lang, output_audio_dir=output_audio_dir)

        # Skip if no transcript was found (returns None)
        if res is None:
            skipped_count += 1
            continue

        results.append(res)

    # Save results to CSV
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    with open(out_csv, "w", encoding="utf-8", newline="") as csvf:
        writer = csv.DictWriter(csvf, fieldnames=[
            "audio", "trimmed_audio", "original_duration_s", "final_duration_s",
            "trim_start_s", "trim_end_s",
            "original_transcript", "corrected_transcript",
            "model_transcript", "was_corrected", "was_audio_trimmed", "correction_reasons",
            "chosen_sim", "final_similarity", "used_skip_first"
        ])
        writer.writeheader()

        for r in results:
            writer.writerow({
                "audio": r["audio"],
                "trimmed_audio": r["trimmed_audio"],
                "original_duration_s": r["original_duration_s"],
                "final_duration_s": r["final_duration_s"],
                "trim_start_s": r["trim_start_s"],
                "trim_end_s": r["trim_end_s"],
                "original_transcript": r["original_transcript"],
                "corrected_transcript": r["corrected_transcript"],
                "model_transcript": r["model_transcript"],
                "was_corrected": r["was_corrected"],
                "was_audio_trimmed": r["was_audio_trimmed"],
                "correction_reasons": r["correction_reasons"],
                "chosen_sim": r["chosen_sim"],
                "final_similarity": r["final_similarity"],
                "used_skip_first": r["used_skip_first"]
            })

    print(f"\n✓ Saved CSV with {len(results)} results: {out_csv}")

    if skipped_count > 0:
        print(f"  ⚠ Skipped {skipped_count} files with no transcript")

    # Print summary statistics
    corrected_count = sum(1 for r in results if r["was_corrected"])
    audio_trimmed_count = sum(1 for r in results if r["was_audio_trimmed"])

    print(f"  - {corrected_count}/{len(results)} transcripts were corrected")
    print(f"  - {audio_trimmed_count}/{len(results)} audio files were trimmed")

    if len(results) > 0:
        avg_original_dur = sum(r['original_duration_s'] for r in results) / len(results)
        avg_final_dur = sum(r['final_duration_s'] for r in results) / len(results)
        print(f"  - Average original duration: {avg_original_dur:.2f}s")
        print(f"  - Average final duration: {avg_final_dur:.2f}s")
        print(f"  - Average time saved per file: {avg_original_dur - avg_final_dur:.2f}s")
        print(f"  - Average final similarity: {sum(r['final_similarity'] for r in results)/len(results):.2%}")

    # Show examples of corrections
    corrections = [r for r in results if r["was_corrected"]]
    if corrections:
        print(f"\n📝 Example corrections:")
        for r in corrections[:3]:  # Show first 3
            filename = Path(r["audio"]).name
            print(f"  • {filename}")
            print(f"    Original: {r['original_transcript'][:60]}...")
            print(f"    Corrected: {r['corrected_transcript'][:60]}...")
            print(f"    Reason: {r['correction_reasons']}")
            print()

if __name__ == "__main__":
    main()

Found 21 audio files in: /content/drive/MyDrive/Anju Project (1)/Audio_segments/May/Konk Prime News_040517
Looking for transcripts in: /content/drive/MyDrive/Anju Project (1)/training_through_bot/May/Konk Prime News_040517
Output will be saved to: /content/drive/MyDrive/verification_results/output.csv
Trimmed audio will be saved to: /content/drive/MyDrive/verification_results/trimmed_audio



Processing audio files:   0%|          | 0/21 [00:00<?, ?it/s]



2025-11-21 17:13:07 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files:  14%|█▍        | 3/21 [00:26<02:39,  8.85s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 17:13:30 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


...

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 17:18:19 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files:  95%|█████████▌| 20/21 [05:37<00:22, 22.86s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps
2025-11-21 17:18:39 - whisperx.alignment - ERROR - No default alignment model for language: mr. Please find a wav2vec2.0 model finetuned on this language at https://huggingface.co/models, then pass the model name via --align_model [MODEL_NAME]


Processing audio files: 100%|██████████| 21/21 [05:57<00:00, 17.03s/it]

  Note: Word-level alignment not available for 'mr', using segment timestamps

✓ Saved CSV with 15 results: /content/drive/MyDrive/verification_results/output.csv
  ⚠ Skipped 6 files with no transcript
  - 14/15 transcripts were corrected
  - 14/15 audio files were trimmed
  - Average original duration: 23.37s
  - Average final duration: 18.26s
  - Average time saved per file: 5.12s
  - Average final similarity: 6.98%

📝 Example corrections:
  • Konk Prime News_040517_segment_003.mp3
    Original: ओला उबेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वोस गोंयकार टॅ...
    Corrected: ओला उबेराचेर कॅम्पेनीन गोंयकार टॅक्सीवाले नेर्वोस गोंयकार टॅ...
    Reason: Last word mismatch: 'इशू' vs 'awaits' (score: 0.0) | Removed half-spoken last word: 'इशू' | Audio trimmed: 0.19s to 0.19s (from 24.96s)

  • Konk Prime News_040517_segment_004.mp3
    Original: सरकार व्हरतलो केंद्रामेरेन एक्शन घेवप गोंयचे अधिकारकक्षेंत न...
    Corrected: सरकार व्हरतलो केंद्रामेरेन एक्शन घेवप गोंयचे अधिकारकक्षेंत न...
    R

In [ ]:
!pip install --no-cache-dir -U transformers==4.41.0 accelerate==0.30.0 datasets==2.19.0 librosa jiwer sentencepiece evaluate


In [ ]:
import pandas as pd
from datasets import Dataset, Audio

CSV_PATH = "/content/drive/MyDrive/verification_results/output.csv"

df = pd.read_csv(CSV_PATH)

# Use corrected transcript column
df = df[["trimmed_audio", "corrected_transcript"]].rename(
    columns={"trimmed_audio": "audio", "corrected_transcript": "text"}
)

# Drop rows with missing files (just in case)
df = df[df["audio"].notna() & df["text"].notna()]

dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

dataset


Dataset({
    features: ['audio', 'text', '__index_level_0__'],
    num_rows: 14
})

In [ ]:
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)

MODEL_NAME = "openai/whisper-small"

feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(
    MODEL_NAME,
    language="Marathi",
    task="transcribe"
)
processor = WhisperProcessor.from_pretrained(
    MODEL_NAME,
    language="Marathi",
    task="transcribe"
)

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

# Disable English-only mode → allows Marathi/Konkani
model.config.forced_decoder_ids = tokenizer.get_decoder_prompt_ids(
    language="Marathi", task="transcribe"
)
model.config.suppress_tokens = []


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
def preprocess(batch):
    audio = batch["audio"]

    # Extract log-mel spectrogram
    inputs = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    )
    batch["input_features"] = inputs["input_features"][0]

    # Tokenize text
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

dataset = dataset.map(preprocess)


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/whisper-small-konkani",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=10,
    fp16=True,
    evaluation_strategy="no",
    save_strategy="epoch",
    logging_steps=50,
    push_to_hub=False,
)


RuntimeError: Failed to import transformers.trainer_seq2seq because of the following error (look up to see its traceback):
cannot import name 'clear_device_cache' from 'accelerate.utils.memory' (/usr/local/lib/python3.12/dist-packages/accelerate/utils/memory.py)

In [ ]:
"""
Fine-tune Whisper model on corrected Marathi transcripts

This script:
1. Loads your corrected CSV data (audio + transcript pairs)
2. Prepares dataset for Whisper fine-tuning
3. Fine-tunes Whisper model using Hugging Face transformers
4. Saves the fine-tuned model

Requirements:
pip install transformers datasets accelerate evaluate jiwer tensorboard
"""

import os
import csv
import torch
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import librosa
from datasets import Dataset, DatasetDict, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate

# ============================================
# CONFIGURATION - CHANGE THESE PATHS
# ============================================

# Path to your CSV file from the verification script
CSV_PATH = "/content/drive/MyDrive/verification_results/output.csv"

# Path to directory containing trimmed audio files
AUDIO_DIR = "/content/drive/MyDrive/verification_results/trimmed_audio"

# Output directory for fine-tuned model
OUTPUT_DIR = "/content/drive/MyDrive/whisper_marathi_finetuned"

# Base Whisper model to fine-tune (options: tiny, base, small, medium, large)
BASE_MODEL = "openai/whisper-small"

# Language code
LANGUAGE = "mr"  # Marathi
TASK = "transcribe"

# Training hyperparameters
BATCH_SIZE = 8  # Reduce if you get OOM errors
LEARNING_RATE = 1e-5
NUM_EPOCHS = 3
WARMUP_STEPS = 500
EVAL_STEPS = 500
SAVE_STEPS = 500
GRADIENT_ACCUMULATION_STEPS = 2

# ============================================
# LOAD AND PREPARE DATASET
# ============================================

def load_dataset_from_csv(csv_path, audio_dir):
    """
    Load corrected transcripts from CSV and prepare for training
    """
    print(f"Loading data from {csv_path}...")

    df = pd.read_csv(csv_path)

    # Filter: only use corrected transcripts with trimmed audio
    df = df[df['was_audio_trimmed'] == True].copy()

    if len(df) == 0:
        print("Warning: No trimmed audio files found in CSV!")
        # Fallback: use all files with transcripts
        df = pd.read_csv(csv_path)
        df = df[df['corrected_transcript'].notna()].copy()

    print(f"Found {len(df)} audio-transcript pairs")

    # Prepare data in the format needed
    data = []
    audio_dir = Path(audio_dir)

    for idx, row in df.iterrows():
        # Use trimmed audio if available, otherwise original
        audio_path = row['trimmed_audio'] if row['trimmed_audio'] else row['audio']
        audio_path = Path(audio_path)

        # Check if file exists
        if not audio_path.exists():
            # Try looking in the audio_dir
            audio_path = audio_dir / audio_path.name

        if not audio_path.exists():
            print(f"Warning: Audio file not found: {audio_path}")
            continue

        transcript = row['corrected_transcript'].strip()

        if transcript:  # Only include non-empty transcripts
            data.append({
                'audio': str(audio_path),
                'sentence': transcript,
                'duration': row.get('final_duration_s', 0)
            })

    print(f"Loaded {len(data)} valid samples")

    # Create HuggingFace dataset
    dataset = Dataset.from_list(data)

    # Cast audio column to Audio type (this will handle loading)
    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

    # Split into train/validation (90/10)
    train_test_split = dataset.train_test_split(test_size=0.1, seed=42)

    return DatasetDict({
        'train': train_test_split['train'],
        'validation': train_test_split['test']
    })

# ============================================
# DATA COLLATOR
# ============================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator for Whisper fine-tuning
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # If bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# ============================================
# PREPROCESSING
# ============================================

def prepare_dataset(batch, processor):
    """
    Preprocess audio and text for Whisper
    """
    # Load and preprocess audio
    audio = batch["audio"]

    # Compute log-Mel input features
    batch["input_features"] = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids

    return batch

# ============================================
# METRICS
# ============================================

def compute_metrics(pred, processor, tokenizer):
    """
    Compute Word Error Rate (WER) metric
    """
    wer_metric = evaluate.load("wer")

    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad token id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# ============================================
# MAIN TRAINING FUNCTION
# ============================================

def main():
    print("="*60)
    print("WHISPER MARATHI FINE-TUNING")
    print("="*60)

    # Check CUDA availability
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\nUsing device: {device}")
    if device == "cuda":
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # Load dataset
    print("\n" + "="*60)
    print("STEP 1: Loading Dataset")
    print("="*60)
    dataset = load_dataset_from_csv(CSV_PATH, AUDIO_DIR)

    print(f"\nTrain samples: {len(dataset['train'])}")
    print(f"Validation samples: {len(dataset['validation'])}")

    # Show example
    print("\nExample sample:")
    print(f"  Audio: {dataset['train'][0]['audio']['path']}")
    print(f"  Transcript: {dataset['train'][0]['sentence'][:100]}...")

    # Load Whisper model components
    print("\n" + "="*60)
    print("STEP 2: Loading Whisper Model")
    print("="*60)
    print(f"Base model: {BASE_MODEL}")

    feature_extractor = WhisperFeatureExtractor.from_pretrained(BASE_MODEL)
    tokenizer = WhisperTokenizer.from_pretrained(BASE_MODEL, language=LANGUAGE, task=TASK)
    processor = WhisperProcessor.from_pretrained(BASE_MODEL, language=LANGUAGE, task=TASK)

    model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []

    # Set language and task for generation
    model.generation_config.language = LANGUAGE
    model.generation_config.task = TASK

    # Preprocess dataset
    print("\n" + "="*60)
    print("STEP 3: Preprocessing Dataset")
    print("="*60)

    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor),
        remove_columns=dataset.column_names["train"],
        num_proc=1
    )

    print("Preprocessing complete!")

    # Data collator
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )

    # Training arguments
    print("\n" + "="*60)
    print("STEP 4: Setting Up Training")
    print("="*60)

    training_args = Seq2SeqTrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_EPOCHS,
        evaluation_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_strategy="steps",
        save_steps=SAVE_STEPS,
        logging_steps=100,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
        gradient_checkpointing=True,  # Save memory
        predict_with_generate=True,
        generation_max_length=225,
    )

    print(f"\nTraining configuration:")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
    print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
    print(f"  Learning rate: {LEARNING_RATE}")
    print(f"  Epochs: {NUM_EPOCHS}")
    print(f"  Mixed precision: {training_args.fp16}")

    # Initialize trainer
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=lambda pred: compute_metrics(pred, processor, tokenizer),
        tokenizer=processor.feature_extractor,
    )

    # Start training
    print("\n" + "="*60)
    print("STEP 5: Training Model")
    print("="*60)
    print("\nStarting training... This will take a while!")
    print("Monitor progress with: tensorboard --logdir=" + OUTPUT_DIR)

    trainer.train()

    # Save final model
    print("\n" + "="*60)
    print("STEP 6: Saving Model")
    print("="*60)

    trainer.save_model(OUTPUT_DIR)
    processor.save_pretrained(OUTPUT_DIR)

    print(f"\n✅ Model saved to: {OUTPUT_DIR}")

    # Evaluate on validation set
    print("\n" + "="*60)
    print("STEP 7: Final Evaluation")
    print("="*60)

    metrics = trainer.evaluate()

    print("\nFinal Metrics:")
    for key, value in metrics.items():
        print(f"  {key}: {value:.4f}")

    print("\n" + "="*60)
    print("TRAINING COMPLETE! 🎉")
    print("="*60)
    print(f"\nYour fine-tuned model is saved at: {OUTPUT_DIR}")
    print("\nTo use the model:")
    print(f"""
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

processor = WhisperProcessor.from_pretrained("{OUTPUT_DIR}")
model = WhisperForConditionalGeneration.from_pretrained("{OUTPUT_DIR}")

# Transcribe audio
audio_path = "your_audio.wav"
# ... (load and process audio)
    """)

if __name__ == "__main__":
    main()

RuntimeError: Failed to import transformers.trainer_seq2seq because of the following error (look up to see its traceback):
cannot import name 'clear_device_cache' from 'accelerate.utils.memory' (/usr/local/lib/python3.12/dist-packages/accelerate/utils/memory.py)

In [ ]:
!pip install datasets==2.18.0
!pip install transformers==4.41.0
!pip install accelerate==0.30.1
!pip install evaluate jiwer librosa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
^C
